In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import firm_revised


In [34]:
seed = 100

In [35]:
df = pd.read_csv(f'../data/market_{seed}.csv')
print(df)

        price      cost  product  time      Car1      Car2    mshare  \
0    3.641920  1.386685        1     1  5.543405  5.891322  0.077929   
1    3.590798  1.358202        2     1  5.278369  5.209202  0.111678   
2    3.561183  1.342556        3     1  5.424518  5.185328  0.105947   
3    3.666598  1.399770        4     1  5.844776  5.108377  0.083860   
4    3.568812  1.345970        5     1  5.004719  5.219697  0.128910   
..        ...       ...      ...   ...       ...       ...       ...   
985  3.085907  1.248011        6    99  5.121569  5.978624  0.088213   
986  2.980473  1.182043        7    99  5.670749  5.811683  0.074662   
987  2.966102  1.172727        8    99  5.825853  5.171941  0.084378   
988  2.917306  1.141750        9    99  5.136707  5.816225  0.100868   
989  2.894211  1.127399       10    99  5.575093  5.274074  0.096568   

      profits   markups  
0    0.175749  1.626350  
1    0.249331  1.643788  
2    0.235057  1.652539  
3    0.190096  1.619429  
4    

In [36]:
# All the parameters of interest 
beta = np.array([2, -0.5, -0.3])
mu = 0.5
omega = 1


# The number of firms in the market
J = 10

# The number of product characteristics 
K = 2

# The number of consumers in the market 
N = 500

# The number of period that I want to run the market for 
T = 100

In [37]:
# Unpacking the X and the price
X1 = np.array(df['Car1'][:J])
X2 = np.array(df['Car2'][:J])
all_X = np.column_stack((X1, X2))
X0 = np.ones(J)
X = np.column_stack((X0, all_X))

eq_price = np.array(df['price'])

true_share = np.array(df['mshare'])
# print(true_share)

all_est_shares = np.zeros(J*(T-1))
# print(all_est_shares)


In [38]:
# The idea is that you want to min the distance between the true share and the made up share by optimizing the fiven vector theta that determines the share 

def f(theta, true_share, N, J, X, price):
    beta = theta[:K+1]
    mu = theta[K+1]
    omega = theta[K+2]  
    # make a numpy array all estimated shares 
    for t in range(0, T-1):
        eq_price_slice = price[t*J:(t+1)*J]
        v_p = np.random.normal(0, 1, N)
        e = np.random.gumbel(0, 1, N*J)
        shares_per_firm = firm_revised.share(N, J, X, v_p, eq_price_slice, beta, mu, omega, e)[0]
        all_est_shares[t*J:(t+1)*J] = shares_per_firm
    return np.linalg.norm(true_share - all_est_shares)

In [ ]:
v_p = np.random.normal(0, 1, N)
e = np.random.gumbel(0, 1, N*J)
shares_estimation = firm_revised.share(N, J, X, v_p, eq_price_slice, beta, mu, omega, e)[0]

In [39]:
# theta_0 = np.array([2., -0.5, -0.3, 0.5, 1.])
theta_0 = np.array([1.5, -1., -1., 0.4, 1.])


In [40]:
res = minimize(f, theta_0, args=(true_share, N, J, X, eq_price), method = 'Nelder-Mead')
optim_vec = res.x
print(optim_vec)

ValueError: operands could not be broadcast together with shapes (5000,) (500,) 

In [ ]:
# Retriving the estimated market shares at the estimated parametre 


In [ ]:
all_val_b0 = []
all_val_b1 = []
all_val_b2 = []
all_val_mu = []
all_val_omega = []

for l in range(1, 10):
    
    res = minimize(f, theta_0, args=(true_share, N, J, X, eq_price), method = 'Nelder-Mead')
    optim_vec = res.x

    all_val_b0.append(optim_vec[0])
    all_val_b1.append(optim_vec[1])
    all_val_b2.append(optim_vec[2])
    all_val_mu.append(optim_vec[3])
    all_val_omega.append(optim_vec[4])

print(all_val_b0)
print(all_val_b1)
print(all_val_b2)
print(all_val_mu)
print(all_val_omega)


KeyboardInterrupt: 